In [ ]:
import  os
import  cv2
import  shutil
import  tqdm
import  natsort
import  subprocess
import  numpy               as      np
import  matplotlib.pyplot   as      plt

from    skimage.measure     import  ransac, LineModelND

In [ ]:
def get_subdirectories(root_dir, max_depth=2):
    directories = []
    for root, dirs, _ in sorted(os.walk(root_dir)):
        if root == root_dir:
            continue  # Skip the root directory itself
        depth = root[len(root_dir):].count(os.sep)
        if depth < max_depth:
            directories.append(root)
        else:
            del dirs[:]  # Stop descending further
    return directories

def load_files(ad):
    valid_extensions = {"tiff", "tif", "png", "jpg", "jpeg", "bmp", "gif", "webp"}  # Common image formats
    FileNames = []
    for file in sorted(os.listdir(ad)):
        try:
            if file.split(".")[-1].lower() in valid_extensions:
                FileNames.append(file)
        except IndexError:
            pass
    return natsort.natsorted(FileNames)

def img_mkr(experiment):
    subprocess.run([
                "ffmpeg",
                
                "-hide_banner",
                "-loglevel","error",

                "-i", f"./{experiment}/{experiment.split('/')[-1]}.mp4",
                "-vf", "pad=1280:1024:0:872:white",  # Halve width and height "fps=240,scale=iw/4:ih/2"
                "-q:v", "2",
                f"{experiment}/%06d.jpg"])
    

def img_mkr_15(experiment):
    subprocess.run([
                "ffmpeg",
                
                "-hide_banner",
                "-loglevel", "error",

                "-i", f"./{experiment}/{experiment.split('/')[-1]}.mp4",
                "-vf", "pad=1280:1024:0:872:white,select='lt(n\,15)'",  # Extract only the first 15 frames
                "-vsync", "vfr",  # Ensure variable frame rate handling
                "-q:v", "2",
                f"{experiment}/%06d.jpg"
    ])


def img_mkr_rotation(experiment, N, angle):
    subprocess.run([
                "ffmpeg",
                "-hide_banner",
                "-loglevel", "error",
                "-i", f"./{experiment}/{experiment.split('/')[-1]}.mp4",
                "-vf", f"crop=1280:ih-{N}:0:0,rotate={angle}*(PI/180):ow=rotw({angle}*(PI/180)):oh=roth({angle}*(PI/180)):c=white,select='lt(n\\,15)'",
                "-vsync", "vfr",
                "-q:v", "2",
                f"{experiment}/%06d.jpg"
    ])




def fit_and_rotate_image(image_path,experiment:str=None,
                         results:bool=False):
    # Load image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Detect edges using Canny
    edges = cv2.Canny(image, 50, 150)
    
    # Get edge coordinates
    y_indices, x_indices = np.where(edges > 0)
    points = np.column_stack((x_indices, y_indices))
    
    # Apply RANSAC to fit a line
    model, inliers = ransac(points, LineModelND, min_samples=2, residual_threshold=2, max_trials=1000)
    
    # Get line parameters
    line_x = np.array([min(x_indices), max(x_indices)])
    line_y = model.predict_y(line_x)
    
    # Compute angle of rotation
    dx = line_x[1] - line_x[0]
    dy = line_y[1] - line_y[0]
    angle = np.degrees(np.arctan2(dy, dx))
    # Rotate image to level the line
    (h, w) = image.shape
    center = (w // 2, h // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=255)
    
    # Crop the lower part of the image, keeping only the top 5 rows
    rotated_image = rotated_image[:, :]
    
    if results:
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(image, cmap='gray')
        plt.plot(line_x, line_y, color='red', linewidth=2, label='Fitted line')
        plt.legend()
        plt.title(fr"{experiment} with Fitted Line")
        
        plt.subplot(1, 2, 2)
        plt.imshow(rotated_image, cmap='gray')
        plt.title("Rotated and Cropped Image (Leveled Surface)")
        _folder = os.path.join(os.path.split(image_path)[0],"rotation")
        if not os.path.exists(_folder):
            os.mkdir(_folder, exist_ok=True )
        plt.savefig(os.path.join(_folder,"result.png"),dpi=300)
        plt.close()
    
    return angle, image.shape, rotated_image

def fit_image(image, black_base_line=10):
    
    # Detect edges using Canny
    edges = cv2.Canny(image, 50, 150)
    
    # Get edge coordinates
    y_indices, x_indices = np.where(edges > 0)
    points = np.column_stack((x_indices, y_indices))
    
    # Apply RANSAC to fit a line
    model, inliers = ransac(points, LineModelND, min_samples=2, residual_threshold=2, max_trials=1000)
    
    # Get line parameters
    line_x = np.array([min(x_indices), max(x_indices)])
    line_y = model.predict_y(line_x)
    
    # Compute angle of rotation
    dx = line_x[1] - line_x[0]
    dy = line_y[1] - line_y[0]
    angle = np.degrees(np.arctan2(dy, dx))
    return  int((line_y[1]+line_y[0])//2) - black_base_line

In [ ]:
# frame extraction
for tilt in get_subdirectories(r"Bubble"):
    for experiment in tqdm.tqdm(get_subdirectories(tilt)):
        # img_mkr(experiment) #img_mkr_15(experiment)

        #for finding the rotation angle
        files = load_files(experiment)
        # #for standardizing base line
        # angle,_shape, rotated_image = fit_and_rotate_image(os.path.join(experiment,files[10]),results=True)
        
        # lower_rows = 20
        # for file in files:
        #     # Load image in grayscale
        #     image = cv2.imread(os.path.join(experiment,file), cv2.IMREAD_GRAYSCALE)
        #     (h, w) = image.shape
        #     center = (w // 2, h // 2)
        #     rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
        #     rotated_image = cv2.warpAffine(image, rotation_matrix, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=255)
        #     rotated_image[fit_image(rotated_image)+lower_rows:,:] = 0
        #     cv2.imwrite(os.path.join(experiment,file), rotated_image[:fit_image(rotated_image)+lower_rows,30:-5])
        # break
        for file in reversed(files):
            image = cv2.imread(os.path.join(experiment,file), cv2.IMREAD_GRAYSCALE)

            if np.sum(image[962,:]>=200)== image.shape[1]:
                pass
                # print("drop is not in the canvas")

            elif np.sum(image[972,:150]>=200)>30 or np.sum(image[972,:40]<=100)>10:
                pass
                # print("partially inside canvas")
            else:
                plt.imshow(image, "grey")
                plt.title(f"{experiment}")
                plt.show()
                break
        # break
    break

In [ ]:
np.where(image[972,:]<=50)

In [ ]:
for file in files[1:]:
    img = cv2.imread(os.path.join(experiment,file))
    img.shape

    if (img[992,1240,:]>np.array([200,200,200])).sum()==3:
        break
    else:
        os.remove(os.path.join(experiment,file))

shutil.copy("Bubble/000000.jpg",f"{experiment}/000000.jpg")

In [ ]:
# # frame extraction
# for tilt in get_subdirectories(r"Bubble"):
#     for experiment in tqdm.tqdm(get_subdirectories(tilt)):        

#         files = load_files(experiment)

#         try:
#             for file in files:
#                 img = cv2.imread(os.path.join(experiment,file))
#                 if (img[991,1243,:]>np.array([200,200,200])).sum()==3:
#                     break
#                 os.remove(os.path.join(experiment,file))

#             for file in reversed(files):
#                 img = cv2.imread(os.path.join(experiment,file))
#                 if (img[978,12,:]>np.array([200,200,200])).sum()==3:
#                     break
#                 os.remove(os.path.join(experiment,file))
#         except:
#             print(experiment)
        
#         shutil.copy("Bubble/000000.jpg",f"{experiment}/000000.jpg")
#         shutil.copytree("Bubble/slope",f"{experiment}/slope",dirs_exist_ok=True)
#         break
#     break

In [ ]:
# # bad frame deletion
# for tilt in get_subdirectories(r"Bubble"):
#     for experiment in tqdm.tqdm(get_subdirectories(tilt)):        
#         files = load_files(experiment)
#         for file in files:
#             img_path = os.path.join(experiment,file)
#             try:
#                 img = cv2.imread(img_path)
#             except:
#                 os.remove(img_path)

# Automatically detecting 

the angle to find drops in equal situation

In [ ]:
# import matplotlib
# matplotlib.use('TkAgg')

In [ ]:
# # bad frame deletion
# for tilt in get_subdirectories(r"Bubble"):
#     for experiment in (get_subdirectories(tilt)):        
#         files = load_files(experiment)
#         fit_and_rotate_image(os.path.join(experiment,files[10]),experiment = tilt.split("/")[-1]+"-"+experiment.split("/")[-1])
#         # break
#     break
